In [7]:
"""
Import packages
"""


import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import fnmatch
import skimage as sk
import scipy as sp
from scipy import ndimage
from skimage import feature
from cnn_functions import get_image
from cnn_functions import format_coord as cf
from skimage import morphology as morph
from skimage.transform import resize
import matplotlib.pyplot as plt
from pywt import WaveletPacket2D

# Define maximum number of training examples
max_training_examples = 10000000
window_size_x = 30
window_size_y = 30

# Load data
direc_name = '/home/vanvalen/DeepCell2/training_data/HeLa_joint'
file_name_save = os.path.join('/home/vanvalen/DeepCell2/training_data_npz/', 'HeLa_all_newnorm_61x61.npz')
training_direcs = ["set1", "set2", "set3", "set4", "set5"] 
channel_names = ["phase", "nuclear"]

In [91]:
# Specify the number of feature masks that are present
num_of_features = 2

# Some of the features that are edges we would like to do morphological
# dilation (disk of radius dil_radius) for data augmentation. This vector identifies 
# which features are edges and which ones arent (1 for an edge, 0 if not an edge)

is_edge_feature = [1,0]
dil_radius = 1

num_direcs = len(training_direcs)
num_channels = len(channel_names)

imglist = []
for direc in training_direcs:
	imglist += os.listdir(os.path.join(direc_name, direc))

# Load one file to get image sizes
img_temp = get_image(os.path.join(direc_name,training_direcs[0],imglist[0]))
image_size_x, image_size_y = img_temp.shape

# Initialize arrays for the training images and the feature masks
channels = np.zeros((num_direcs, num_channels, image_size_x, image_size_y), dtype='float32')
feature_mask = np.zeros((num_direcs, num_of_features + 1, image_size_x, image_size_y))

# Load training images
direc_counter = 0
for direc in training_direcs:
	imglist = os.listdir(os.path.join(direc_name, direc))
	channel_counter = 0

	# Load channels
	for channel in channel_names:
		for img in imglist: 
			if fnmatch.fnmatch(img, r'*' + channel + r'*'):
				print img
				channel_file = img
				channel_file = os.path.join(direc_name, direc, channel_file)
				channel_img = np.float64(get_image(channel_file))

				p50 = np.percentile(channel_img, 50)
				channel_img /= p50

				# channel_img -= np.mean(channel_img)
				#avg_kernel = np.ones((2*window_size_x + 1, 2*window_size_y + 1))
				#channel_img -= ndimage.convolve(channel_img, avg_kernel)/avg_kernel.size
                wp = WaveletPacket2D(data = channel_img, wavelet = 'haar', mode = 'sym')
                noise_1 = resize(np.array(wp['a'*1].data), channel_img.shape, order = 3, mode = 'reflect')/2**1
                noise_2 = resize(np.array(wp['a'*2].data), channel_img.shape, order = 3, mode = 'reflect')/2**2
                noise_3 = resize(np.array(wp['a'*3].data), channel_img.shape, order = 3, mode = 'reflect')/2**3
                detail =  noise_1 - noise_3
                background = resize(np.array(wp['a'*5].data), channel_img.shape, order = 3, mode = 'reflect')/2**5
                channel_img -= noise_1 + noise_2 + background
                channel_img = detail
                channels[direc_counter,channel_counter,:,:] = channel_img
                channel_counter += 1

	direc_counter += 1

phase.png
nuclear.png
phase.png
nuclear.png
phase.png
nuclear.png
phase.png
nuclear.png
phase.png
nuclear.png


In [50]:
direc_counter = 0
for direc in training_direcs:
	# Load feature mask
	for j in xrange(num_of_features):
		feature_name = "feature_" + str(j) + r".*"
		for img in imglist:
			if fnmatch.fnmatch(img, feature_name):
				feature_file = os.path.join(direc_name, direc, img)
				feature_img = get_image(feature_file)

				if np.sum(feature_img) > 0:
					feature_img /= np.amax(feature_img)

				if is_edge_feature[j] == 1:
					strel = sk.morphology.disk(dil_radius)
					feature_img = sk.morphology.binary_dilation(feature_img, selem = strel)

				feature_mask[direc_counter,j,:,:] = feature_img

	# Thin the augmented edges by subtracting the interior features.
	for j in xrange(num_of_features):
		if is_edge_feature[j] == 1:
			for k in xrange(num_of_features):
				if is_edge_feature[k] == 0:
					feature_mask[direc_counter,j,:,:] -= feature_mask[direc_counter,k,:,:]
			feature_mask[direc_counter,j,:,:] = feature_mask[direc_counter,j,:,:] > 0

	# Compute the mask for the background
	feature_mask_sum = np.sum(feature_mask[direc_counter,:,:,:], axis = 0)
	feature_mask[direc_counter,num_of_features,:,:] = 1 - feature_mask_sum

	direc_counter += 1

In [47]:
fig,ax = plt.subplots(len(training_direcs),num_of_features+1, squeeze = False)
print ax.shape
for j in xrange(len(training_direcs)):
	ax[j,0].imshow(channels[j,0,:,:],cmap=plt.cm.gray,interpolation='nearest')
	def form_coord(x,y):
		return cf(x,y,channels[j,0,:,:])
	ax[j,0].format_coord = form_coord
	ax[j,0].axes.get_xaxis().set_visible(False)
	ax[j,0].axes.get_yaxis().set_visible(False)

	for k in xrange(1,num_of_features+1):
		ax[j,k].imshow(feature_mask[j,k-1,:,:],cmap=plt.cm.gray,interpolation='nearest')
		def form_coord(x,y):
			return cf(x,y,feature_mask[j,k-1,:,:])
		ax[j,k].format_coord = form_coord
		ax[j,k].axes.get_xaxis().set_visible(False)
		ax[j,k].axes.get_yaxis().set_visible(False)

plt.show()

(5, 3)


In [94]:
sk.io.imsave('test.tif', channels[4,1,:,:])